In [37]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

#Dataframe with pandas
#df = pd.read_csv('../Python/Data/HMAHCC_COMP.CSV')
#path to Competition file
Comp = pd.read_csv('.../Datasets/Humana/HMAHCC_COMP.csv')
#path to Holdout file
Holdout = pd.read_csv('.../Datasets/Humana/HMAHCC_HOLDOUT.csv')

In [38]:
Holdout.rename(columns={'ID':'id','days':'Days','EVENT_DESCR':'event_descr', 
                        'EVENT_ATTR1' : 'event_attr1', 
                        'EVENT_ATTR2' : 'event_attr2', 
                        'EVENT_ATTR3' : 'event_attr3',
                        'EVENT_ATTR4' : 'event_attr4',
                        'EVENT_ATTR5' : 'event_attr5',
                        'EVENT_ATTR6' : 'event_attr6',
                        'EVENT_ATTR7' : 'event_attr7',
                        'EVENT_ATTR8' : 'event_attr8',
                        'EVENT_ATTR9' : 'event_attr9',
                        'EVENT_ATTR10' : 'event_attr10'}, inplace=True )

In [39]:

class AtRisk:
    
#Danny's Code:    
    def __init__(self,data):
        self.data = data
    
    def LTOT (self, ID):
    #Qualifying Events start at Day 0, easy to find opioid events via MME
        temp = self.data.loc[self.data.id == ID]
        after180 = temp.loc[(~temp.MME.isnull()) & (temp.Days >= -90)]
    #Define series to add
        days = after180.loc[:,'Days']
        add = after180.loc[:,'PAY_DAY_SUPPLY_CNT']
    #generalizable loop
        a = 0
        i = 0
        y = 0
        while i in range(len(days)):            
            #i is current event
            b = i + 1
            c = i - 1
            if b >= len(days):
                return False
            currentday= days.iloc[i]
            nextday = days.iloc[b]
            supply = add.iloc[i]
            previoussupply = days.iloc[c] + add.iloc[c]
            fromcurrent180 = days.iloc[i] + 180
            currentsupply = days.iloc[i] + add.iloc[i]
            nextsupply = days.iloc[b] + add.iloc[b]
            month6begin = days.iloc[a]
            month6end = days.iloc[a] + 180
            overlap = 0
            over180 = 0 

            #Remove days over 180 days, ignore if not in 6 month window
            if (currentsupply >= month6end): 
                over180 = currentsupply - month6end
            #Remove overlapping days
            if (currentday < month6end) & (currentsupply > nextday):
                overlap = currentsupply - nextday
            #90% of 6 month period
            Value = (supply - overlap - over180)
            y += Value
            if y >= 162:
                return True
            else:
                i += 1
                #a goes to next event when 6 month window fails
            if (currentday >= month6end) & (currentday > previoussupply + 90) & (y < 162):
                a = i
                y = 0
            if b >= len(days):
                if (nextsupply >= fromcurrent180) & (nextday < month6end): 
                    over180 = fromcurrent180 - nextsupply
                    y += supply - overlap
                if y < 162:
                    return False
    def PercLTOT (self, ID):
    #Qualifying Events start at Day 0, easy to find opioid events via MME
        temp = self.data.loc[self.data.id == ID]
        after180 = temp.loc[(~temp.MME.isnull()) & (temp.Days >= -90)]
    #Define series to add
        days = after180.loc[:,'Days']
        add = after180.loc[:,'PAY_DAY_SUPPLY_CNT']
    #generalizable loop
        a = 0
        i = 0
        y = 0
        while i in range(len(days)):            
            #i is current event
            b = i + 1
            c = i - 1
            if b >= len(days):
                return y
            currentday= days.iloc[i]
            nextday = days.iloc[b]
            supply = add.iloc[i]
            previoussupply = days.iloc[c] + add.iloc[c]
            fromcurrent180 = days.iloc[i] + 180
            currentsupply = days.iloc[i] + add.iloc[i]
            nextsupply = days.iloc[b] + add.iloc[b]
            month6begin = days.iloc[a]
            month6end = days.iloc[a] + 180
            overlap = 0
            over180 = 0 

            #Remove days over 180 days, ignore if not in 6 month window
            if (currentsupply >= month6end): 
                over180 = currentsupply - month6end
            #Remove overlapping days
            if (currentday < month6end) & (currentsupply > nextday):
                overlap = currentsupply - nextday
            #90% of 6 month period
            Value = (supply - overlap - over180)
            y += Value
            if y >= 162:
                return y
            else:
                i += 1
                #a goes to next event when 6 month window fails
            if (currentday >= month6end) & (currentday > previoussupply + 90) & (y < 162):
                a = i
                y = 0
            if b >= len(days):
                if (nextsupply >= fromcurrent180) & (nextday < month6end): 
                    over180 = fromcurrent180 - nextsupply
                    y += supply - overlap
                if y < 162:
                    return y
    
    def TotalMME (self):
        
        OnlyMME = self.data.loc[:, ['id','Days', 'MME', 'PAY_DAY_SUPPLY_CNT']] 
        NoOverlap = OnlyMME.drop_duplicates()
        Multiply = NoOverlap['PAY_DAY_SUPPLY_CNT']* NoOverlap['MME']
        Total = pd.concat([NoOverlap,Multiply], axis = 1)
        Total.rename(columns= {0:'TotalMME'}, inplace = True)
        Final = Total.iloc[:,[0,4]]
        x = Final.groupby('id').agg('sum')
        
        return x
    
    def AD (self):
        Total = self.TotalMME()
        Opioids = self.data.loc[:, ['id','Days', 'MME', 'PAY_DAY_SUPPLY_CNT']] 
        NoOverlap = Opioids.drop_duplicates()
        feature = NoOverlap.groupby('id').sum(skipna = True)
        Average = Total['TotalMME'] / feature['PAY_DAY_SUPPLY_CNT']
        Average = pd.DataFrame(Average)
        Average.rename(columns={ 0 :'AverageDailyMME'}, inplace=True )
        
        return Average

    def benzofill (self):
        
        event = self.data.loc[:, ['id','event_descr','event_attr1']] 
        paid = event.loc[event.event_descr == 'RX Claim - Paid']
        benzo = paid[paid['event_attr1'].str.contains('BENZO', na = False)]
        benzo = benzo.iloc[:,[0,1]]
        benzo.rename(columns = {'id':'id', 'event_descr': 'BenzoFill'}, inplace = True)
        feature = benzo.groupby('id').count()
        return feature

    def opfill (self):
        
        OnlyMME = self.data.loc[:, ['id','Days','MME', 'event_descr']] 
        notnull = OnlyMME[np.isfinite(OnlyMME['MME'])]
        NoOverlap = notnull.drop_duplicates()
        paid = NoOverlap.loc[NoOverlap.event_descr == 'RX Claim - Paid']
        final = paid.loc[:,['id','event_descr']]
        final.rename(columns = {'id':'id', 'event_descr': 'OpioidFill'}, inplace = True)
        calculate = final.groupby('id').count()
        
        return calculate
    
    def nonOPRX (self):
        Dataselect = self.data.loc[:, ['id','Days','event_attr1','MME', 'event_descr']]
        Paid = Dataselect.loc[Dataselect.event_descr == 'RX Claim - Paid']
        NoOverlap = Paid.drop_duplicates()
        final = NoOverlap.loc[:,['id','event_descr']]
        final.rename(columns = {'id':'id', 'event_descr': 'NonOpioid_Rx'}, inplace = True)
        feature = final.groupby('id').count()
        calculate = self.opfill()
        feature1 = feature['NonOpioid_Rx'] - calculate['OpioidFill']
        feature2 = pd.DataFrame(feature1)
        feature2.rename(columns={ 0 :'NonOpioid_Rx'}, inplace=True )

        return feature2
    
#Any diagnosis can be found for patients:
    def Diagnosis (self, diagnosis, name = 'Diagnosis'):
        Event = self.data.loc[:, ['id','event_descr']]
        select = Event.loc[Event.event_descr == diagnosis]
        select.rename(columns = {'id':'id', 'event_descr': name}, inplace = True)
        feature = select.groupby('id').count()
        return feature

#Ray's Code:
    def AUD(self):
        # Extracting History of AUD

        humana_2_1 = self.data[self.data.event_descr == ('Fully Paid Claim' or 'Surgery' or 'New provider')]#events relevant to AUD/SUD diagnosis

        humana_2_1 = humana_2_1.loc[:,['id','event_attr1']]    #selecting diagnosis only, which is attribute #1

        humana_2_2 = self.data[self.data.event_descr == ('RX Claim - Rejected')]    #pulling out separately due to different attribute

        humana_2_2 = humana_2_2.loc[:,['id','event_attr2']]    #selecting diagnosis, which is attribute #2 for rx claim rejected
        # Create a 2-column dataframe for manipulation

        humana_2_2.rename(columns={"id": "id", "event_attr2": "event_attr1"},inplace = True)
        humana_2 = humana_2_1.append([humana_2_2])
        humana_3 = pd.DataFrame.copy(humana_2)

        humana_2.rename(columns={"id": "id", "event_attr1": "AUD_Incidence"},inplace = True)
        humana_2 = humana_2.fillna('NO VALUE')
        # Scan for alcohol-related records

        humana_2 = humana_2[humana_2["AUD_Incidence"].str.contains("ALCOHOL")]
        humana_22 = humana_2[humana_2["AUD_Incidence"].str.contains("NONALCOHOLIC")] # Exclude NASH etc.

        idx1 = set(humana_2.set_index(['id', "AUD_Incidence"]).index)  # This two lines remove duplicate as well
        idx2 = set(humana_22.set_index(['id', "AUD_Incidence"]).index)

        humana_2 = pd.DataFrame(list(idx1 - idx2), columns=humana_2.columns)    
        # Aggregate records for frequency in the 4-year time period

        feature_2 = humana_2.groupby('id').count()
        
        return feature_2

    def SUD (self):
        # Extracting History of SUD

        humana_s = self.data[self.data.event_descr == ('Fully Paid Claim' or 'New provider' or 'RX Claim - Paid')]#events relevant to SUD diagnosis
        #selecting diagnosis only, which is attribute #1
        humana_s = humana_s.loc[:,['id','event_attr1']]   
        humana_m = pd.DataFrame.copy(humana_s)

        # Create a 2-column dataframe for manipulation

        humana_s.rename(columns={"id": "id", "event_attr1": "SUD_Incidence"},inplace = True)
        humana_s = humana_s.fillna('NO VALUE')

        # Scan for substance-related records

        humana_s = humana_s[humana_s["SUD_Incidence"].str.contains("SUBSTANCE")]
        humana_ss = humana_s[humana_s["SUD_Incidence"].str.contains("NOT DUE TO A SUBSTANCE")] # Exclude misc. cases.
        humana_sss = humana_s[humana_s["SUD_Incidence"].str.contains("BODY SUBSTANCE")] # Exclude body substance diagnoses

        idx1 = set(humana_s.set_index(['id', "SUD_Incidence"]).index)  # These three lines remove duplicate as well
        idx2 = set(humana_ss.set_index(['id', "SUD_Incidence"]).index)
        idx3 = set(humana_sss.set_index(['id', "SUD_Incidence"]).index)

        humana_s = pd.DataFrame(list(idx1 - idx2 - idx3), columns=humana_s.columns)

        # Aggregate records for frequency in the 4-year time period

        feature_s = humana_s.groupby('id').count()
        return feature_s
        # Extracting Mental Health Conditions, similar to above
    def MENTAL(self):
        # Create a 2-column dataframe for manipulation
      
        humana_s = self.data[self.data.event_descr == ('Fully Paid Claim' or 'New provider' or 'RX Claim - Paid')]#events relevant to SUD diagnosis
        #selecting diagnosis only, which is attribute #1
        humana_s = humana_s.loc[:,['id','event_attr1']]   
        humana_m = pd.DataFrame.copy(humana_s)

        humana_m.rename(columns={"id": "id", "event_attr1": "Mental Health Conditions"},inplace = True)
        humana_m = humana_m.fillna('NO VALUE')

        # Scan for disability-related records

        humana_m1 = humana_m[humana_m["Mental Health Conditions"].str.contains("DEPRESSIVE")]
        humana_m2 = humana_m[humana_m["Mental Health Conditions"].str.contains("BIPOLAR")]
        humana_m3 = humana_m[humana_m["Mental Health Conditions"].str.contains("SCHIZOPHRENIA")]
        humana_m4 = humana_m[humana_m["Mental Health Conditions"].str.contains("DEMENTIA")]

        idx1 = set(humana_m1.set_index(['id', "Mental Health Conditions"]).index)  # This two lines remove duplicate as well
        idx2 = set(humana_m2.set_index(['id', "Mental Health Conditions"]).index)
        idx3 = set(humana_m3.set_index(['id', "Mental Health Conditions"]).index)
        idx4 = set(humana_m4.set_index(['id', "Mental Health Conditions"]).index)

        humana_m = pd.DataFrame(list(idx1 - idx2 - idx3 - idx4), columns=humana_m.columns) 

        # Aggregate records for frequency in the 4-year time period

        feature_m = humana_m.groupby('id').count()
        
        return feature_m
    

In [40]:
Patient = AtRisk(Holdout)
TotalMME1 = Patient.TotalMME()
AverageDaily1 = Patient.AD()
BenzoidFill1 = Patient.benzofill()
OpioidFill1 = Patient.opfill()
NonOpioidRx1 = Patient.nonOPRX()
NewProvider1 = Patient.Diagnosis('New provider', name = 'NewDoc')
CPD1 = Patient.Diagnosis('New diagnosis - CPD', name = 'CPD_Diagnosis')
CHF1 = Patient.Diagnosis('New diagnosis - CHF', name = 'CHF_Diagnosis')
Diabetes1 = Patient.Diagnosis('New diagnosis - Diabetes', name = 'Diabt_Diagnosis')
Hypertension1 = Patient.Diagnosis('New diagnosis - Hypertension', name = 'Hyprt_Diagnosis')
CAD1 = Patient.Diagnosis('New diagnosis - CAD', name = 'CAD_Diagnosis')
Top51 = Patient.Diagnosis('New diagnosis - Top 5', name = 'Top5_Diagnosis')
AUD1 = Patient.AUD()
SUD1 = Patient.SUD()
Mental1 = Patient.MENTAL()

In [41]:
stack = pd.concat([TotalMME1,AverageDaily1,BenzoidFill1,OpioidFill1,NonOpioidRx1,NewProvider1,CPD1,CHF1,Diabetes1,Hypertension1,CAD1,Top51,AUD1,SUD1,Mental1], axis = 1, sort = False)

In [43]:
stack.fillna(value = 0, inplace = True)
stack.to_csv('SUD&Mental3.csv')

In [ ]:
stack.fillna(value = 0, inplace = True)
stack.to_csv('HoldoutData.csv)

In [20]:
Patient = AtRisk(Comp)

In [21]:
TotalMME = Patient.TotalMME()
AverageDaily = Patient.AD()
BenzoidFill = Patient.benzofill()
OpioidFill = Patient.opfill()
NonOpioidRx = Patient.nonOPRX()
NewProvider = Patient.Diagnosis('New provider', name = 'NewDoc')
CPD = Patient.Diagnosis('New diagnosis - CPD', name = 'CPD_Diagnosis')
CHF = Patient.Diagnosis('New diagnosis - CHF', name = 'CHF_Diagnosis')
Diabetes = Patient.Diagnosis('New diagnosis - Diabetes', name = 'Diabt_Diagnosis')
Hypertension = Patient.Diagnosis('New diagnosis - Hypertension', name = 'Hyprt_Diagnosis')
CAD = Patient.Diagnosis('New diagnosis - CAD', name = 'CAD_Diagnosis')
Top5 = Patient.Diagnosis('New diagnosis - Top 5', name = 'Top5_Diagnosis')
AUD = Patient.AUD()
Mental = Patient.MENTAL()
SUD = Patient.SUD()

In [30]:
features = pd.concat([TotalMME,AverageDaily,BenzoidFill,OpioidFill,NonOpioidRx,NewProvider,CPD,CHF,Diabetes,Hypertension,CAD,Top5,AUD,SUD,Mental], axis = 1, sort = False)
features.fillna(value = 0, inplace = True)
features.to_csv('Features.csv')

In [32]:
CompFeat = pd.read_csv('.../Humana-Mays-excel/Features.csv')

In [33]:
CompFeat.set_index('id', drop = True, inplace = True)

In [34]:
# Extracting History of Cancer

humana_c_1 = Comp[Comp.event_descr == ('RX Claim - Paid')]#events relevant to cancer diagnosis
humana_c_1 = humana_c_1.loc[:,['id','event_attr6']]    #selecting diagnosis only, which is attribute #6

humana_c_2 = Comp[Comp.event_descr == ('RX Claim - New Drug')]    #pulling out separately due to different attribute

humana_c_2 = humana_c_2.loc[:,['id','event_attr5']]    #selecting diagnosis, which is attribute #5 for rx claim rejected
humana_c_1 = humana_c_1.fillna('NO VALUE')
humana_c_1 = humana_c_1[humana_c_1["event_attr6"].str.contains("CANCER")]

humana_c_2 = humana_c_2.fillna('NO VALUE')
humana_c_2 = humana_c_2[humana_c_2["event_attr5"].str.contains("CANCER")]

# Locating patients with cancer from 2 features above
Cancer = pd.concat([humana_c_1['id'],humana_c_2['id']]).unique()
patient6 = CompFeat.drop(Cancer)


patient6 = patient6.dropna(axis = 0)

patient6.to_csv('FeaturesNoCancer.csv')